[View in Colaboratory](https://colab.research.google.com/github/opooladz/ece_210_final_project/blob/master/210_final_project_convnet.ipynb)

In [0]:
# Import Libs 
import numpy as np
import matplotlib.pyplot as plt
import scipy.special
import scipy.io as sio
import math
#from  scipy.ndimage import convolve1d

# Reloading any code written in external .py files.
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
# Keras try model 
def try_model(model, data, name, epochs=200, verbose=1):  
  model.compile(optimizer='nadam',
            loss='categorical_crossentropy',
            metrics=[ 'accuracy'])

  model.summary()  
  history = model.fit(data['x_train'], data['y_train'], 
            epochs=epochs, batch_size=64, verbose=verbose, 
            validation_data=(data['x_val'], data['y_val']))  
  score = model.evaluate(data['x_test'], data['y_test'])
  print('Test accuracy: {}'.format(score))
  
#   out_path = 'serialized_models/{}.h5'.format(name)  
#   model.save(out_path)
  
#   with open('serialized_models/{}_history.pickle'.format(name), 'wb') as handle:
#     pickle.dump(history.history, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
  return history

In [0]:
# Comms Stuff Will add more later
def qammod(m,y):
  if(np.all(y>=m)):
    print('qammod: all elements of X must be in range [0,M-1]')
  
  if (~np.all(y == np.fix(y) ) ):
    print("qammod: all elements of X must be integers");
  
  c = np.sqrt(m);
  if( ~(np.all(c == np.fix(c)) and np.all(np.math.log(c, 2) == np.fix(np.math.log(c, 2))) ) ):
    print("qammod: M must be a square of a power of 2");
  
  b = -2*np.fmod(y,c) + c - 1.;
  a = 2*np.floor(y *1./c) - c + 1.;
  x = a + 1.*1j*b
  # lets hold x in a np array([real,imag])
  # x = np.array([a,1.*b]).T
  return [x , y] 

def qammod2(m,size,trials):
  """
  m - highest integer for the randint
  """
  y = np.random.randint(m,size=(trials,size))
  if(np.all(y>=m)):
    print('qammod: all elements of X must be in range [0,M-1]')
  
  if (~np.all(y == np.fix(y) ) ):
    print("qammod: all elements of X must be integers");
  
  c = np.sqrt(m);
  if( ~(np.all(c == np.fix(c)) and np.all(np.math.log(c, 2) == np.fix(np.math.log(c, 2))) ) ):
    print("qammod: M must be a square of a power of 2");
  
  b = -2*np.fmod(y,c) + c - 1.;
  a = 2*np.floor(y *1./c) - c + 1.;
  #x = a + 1.*1j*b
  # lets hold x in a dp array([real,imag])
  #x = np.array([a,1.*b])
  x = np.dstack((a,b))
  return [x , y]                        

def channel2(m,sequenceLen,trials,Noise,SNRdB,chanelLen):
  
  [x,y] = qammod2(m,sequenceLen,trials)
  
  h = [1., 0.5, 0.3, -0.13, 0.2, 0, 0.4]
  #h = np.random.randn(1,chanelLen)
  #h = [1]
  #print(x.shape)
#   xh = np.empty((sequenceLen+len(h)-1,2))
#   xh[:,0] = np.convolve(x[:,0],h)   
#   xh[:,1] = np.convolve(x[:,1],h)   
  #xh = convolve1d(x,h,axis = 1)
  xh = np.apply_along_axis(lambda m: np.convolve(m,h[0:chanelLen],mode="full"),axis = 1,arr=x)
  #print(xh.shape)
  SNR = 10**(SNRdB/10)
  sym_noise_pow = np.var(xh)/SNR
  sym_noise_scale_fact = np.sqrt(sym_noise_pow/2)
  if(Noise):
    AWGN = sym_noise_scale_fact*np.random.normal(1, size=xh.shape) 
    xh = xh + AWGN
  #return [xhN.T , np.repeat(y,2) ]
  return [xh ,y ]
  
                        
data={}
def generateData2(m,sequenceLen,trials,Noise,SNRdB,chanelLen):
  [data['x_train'],data['y_train']] = channel2(m=m,sequenceLen=sequenceLen,trials=trials,Noise=Noise,SNRdB=SNRdB,chanelLen=chanelLen)
  [data['x_val'],data['y_val']] = channel2(m=m,sequenceLen=sequenceLen,trials=int(trials*0.5),Noise= Noise,SNRdB=SNRdB,chanelLen=chanelLen)
  [data['x_test'],data['y_test']] = channel2(m=m,sequenceLen=sequenceLen,trials=trials,Noise=Noise,SNRdB=SNRdB,chanelLen=chanelLen)

In [18]:
# Visulizing the data

m = 4         # order of modulation
#seqLen = 64    # length of one trial must be square powers of 2 to make sense see below 
trials = 1000   # number of trials 
Noise = 1
SNRdB = 25
chanelLen = 4


x = 4
square_power_of_two = 2**(2*x)
seqLen = square_power_of_two  
print(seqLen)

generateData2(m,seqLen,trials,Noise,SNRdB,chanelLen)
print(data['x_train'].shape) # data  # axis 1 should be 2x seqLen because of conv with a 2tap filter
print(data['y_train'].shape) # labels
#print(data['x_train']) # so u can see the data 

# tmp = (data['x_train'][:,:-1].reshape(trials,seqLen))

# #plot 1
# plt.scatter(np.real(tmp[0][:]),np.imag(tmp[0][:]))   
#                                          # I only plot the first 10 points from 
#                                          # the sequence because as time goes on 
#                                          # the data becomes less dependent on the
#                                          # previous symbols. Even 10 is a lot. 
# plt.title('4-QAM Mod With Multi-Channel AWGN plot 1')
# plt.show()

#plot 2
# Just a pure 16 qam with 1 point on each constalation point 
# clearly we have 16 classes
# [xtmp,ytmp]=  qammod(16,data['y_train'])
# plt.scatter(xtmp[:,0],xtmp[:,1])
# plt.title('16-QAM Mod plot 2')
# plt.show()



256
(1000, 259, 2)
(1000, 256)


In [19]:
# Some keras stuffs
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, Flatten, AveragePooling2D, MaxPooling2D, Dropout, BatchNormalization
from keras.layers import Reshape, Permute
from keras import regularizers
import keras
ykeys = {'y_train','y_val','y_test'}
xkeys = {'x_train','x_val','x_test'}

  
y_train = keras.utils.to_categorical(data['y_train'] , num_classes=m)
y_val = keras.utils.to_categorical(data['y_val'] , num_classes=m)
y_test = keras.utils.to_categorical(data['y_test'] , num_classes=m)

print(data['x_train'].shape)
print(data['x_test'].shape)
print(y_train.shape)
print(data['x_val'].shape)


default_data = {
  'x_train': data['x_train'],
  'x_val': data['x_val'],
  'x_test': data['x_test'],
  'y_train': y_train,
  'y_val': y_val,
  'y_test': y_test,
}


(1000, 259, 2)
(1000, 259, 2)
(1000, 256, 4)
(500, 259, 2)


In [20]:
from keras.layers import Reshape,Dense,Embedding,Conv1D, LSTM,CuDNNLSTM, TimeDistributed, GaussianNoise, Softmax,Flatten
from keras import regularizers

modCNN = Sequential()
modCNN.add(Conv1D(90,kernel_size=(80), input_shape=(seqLen+chanelLen-1,2) ))
modCNN.add(BatchNormalization())
modCNN.add(Conv1D(90,kernel_size=(80), input_shape=(seqLen+chanelLen-1,2) ))
modCNN.add(BatchNormalization())
modCNN.add(Flatten())
modCNN.add(Dense(seqLen*m,bias_regularizer=regularizers.l2(0.005)))
modCNN.add(Reshape(target_shape = (seqLen,m)))
modCNN.add(BatchNormalization())
#modCNN.add(Dropout(0.4))
modCNN.add(Softmax(axis=2))
try_model(modCNN, default_data, 'conv_lstmx', epochs=200)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 180, 90)           14490     
_________________________________________________________________
batch_normalization_7 (Batch (None, 180, 90)           360       
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 101, 90)           648090    
_________________________________________________________________
batch_normalization_8 (Batch (None, 101, 90)           360       
_________________________________________________________________
flatten_3 (Flatten)          (None, 9090)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              9309184   
_________________________________________________________________
reshape_3 (Reshape)          (None, 256, 4)            0         
__________

1000/1000 [==============================] - 1s 848us/step - loss: 0.4206 - acc: 0.9606 - val_loss: 0.6258 - val_acc: 0.8095
Epoch 6/200
1000/1000 [==============================] - 1s 848us/step - loss: 0.3862 - acc: 0.9711 - val_loss: 0.5934 - val_acc: 0.8207
Epoch 7/200
1000/1000 [==============================] - 1s 855us/step - loss: 0.3591 - acc: 0.9785 - val_loss: 0.5655 - val_acc: 0.8298
Epoch 8/200
1000/1000 [==============================] - 1s 858us/step - loss: 0.3360 - acc: 0.9834 - val_loss: 0.5440 - val_acc: 0.8351
Epoch 9/200
1000/1000 [==============================] - 1s 869us/step - loss: 0.3157 - acc: 0.9868 - val_loss: 0.5259 - val_acc: 0.8406
Epoch 10/200
1000/1000 [==============================] - 1s 852us/step - loss: 0.2980 - acc: 0.9899 - val_loss: 0.5107 - val_acc: 0.8444
Epoch 11/200
1000/1000 [==============================] - 1s 869us/step - loss: 0.2820 - acc: 0.9921 - val_loss: 0.4973 - val_acc: 0.8475
Epoch 12/200
1000/1000 [===========================

1000/1000 [==============================] - 1s 848us/step - loss: 0.2213 - acc: 0.9973 - val_loss: 0.4563 - val_acc: 0.8573
Epoch 17/200
1000/1000 [==============================] - 1s 873us/step - loss: 0.2116 - acc: 0.9977 - val_loss: 0.4507 - val_acc: 0.8591
Epoch 18/200
1000/1000 [==============================] - 1s 865us/step - loss: 0.2030 - acc: 0.9980 - val_loss: 0.4455 - val_acc: 0.8594
Epoch 19/200
1000/1000 [==============================] - 1s 859us/step - loss: 0.1947 - acc: 0.9983 - val_loss: 0.4415 - val_acc: 0.8605
Epoch 20/200
1000/1000 [==============================] - 1s 861us/step - loss: 0.1870 - acc: 0.9987 - val_loss: 0.4377 - val_acc: 0.8612
Epoch 21/200
1000/1000 [==============================] - 1s 866us/step - loss: 0.1798 - acc: 0.9989 - val_loss: 0.4338 - val_acc: 0.8624
Epoch 22/200
1000/1000 [==============================] - 1s 863us/step - loss: 0.1727 - acc: 0.9990 - val_loss: 0.4311 - val_acc: 0.8627
Epoch 23/200
1000/1000 [=======================

1000/1000 [==============================] - 1s 869us/step - loss: 0.1440 - acc: 0.9995 - val_loss: 0.4185 - val_acc: 0.8646
Epoch 28/200
1000/1000 [==============================] - 1s 869us/step - loss: 0.1390 - acc: 0.9997 - val_loss: 0.4161 - val_acc: 0.8663
Epoch 29/200
1000/1000 [==============================] - 1s 868us/step - loss: 0.1345 - acc: 0.9997 - val_loss: 0.4141 - val_acc: 0.8668
Epoch 30/200
1000/1000 [==============================] - 1s 853us/step - loss: 0.1299 - acc: 0.9997 - val_loss: 0.4127 - val_acc: 0.8660
Epoch 31/200
1000/1000 [==============================] - 1s 856us/step - loss: 0.1258 - acc: 0.9998 - val_loss: 0.4107 - val_acc: 0.8662
Epoch 32/200
1000/1000 [==============================] - 1s 866us/step - loss: 0.1218 - acc: 0.9998 - val_loss: 0.4092 - val_acc: 0.8670
Epoch 33/200
1000/1000 [==============================] - 1s 849us/step - loss: 0.1180 - acc: 0.9999 - val_loss: 0.4078 - val_acc: 0.8673
Epoch 34/200
1000/1000 [=======================

1000/1000 [==============================] - 1s 875us/step - loss: 0.1014 - acc: 0.9999 - val_loss: 0.4009 - val_acc: 0.8679
Epoch 39/200
1000/1000 [==============================] - 1s 863us/step - loss: 0.0987 - acc: 1.0000 - val_loss: 0.3995 - val_acc: 0.8687
Epoch 40/200
1000/1000 [==============================] - 1s 851us/step - loss: 0.0957 - acc: 1.0000 - val_loss: 0.3984 - val_acc: 0.8686
Epoch 41/200
1000/1000 [==============================] - 1s 855us/step - loss: 0.0932 - acc: 1.0000 - val_loss: 0.3971 - val_acc: 0.8686
Epoch 42/200
1000/1000 [==============================] - 1s 882us/step - loss: 0.0906 - acc: 1.0000 - val_loss: 0.3963 - val_acc: 0.8684
Epoch 43/200
1000/1000 [==============================] - 1s 873us/step - loss: 0.0882 - acc: 1.0000 - val_loss: 0.3954 - val_acc: 0.8683
Epoch 44/200
1000/1000 [==============================] - 1s 842us/step - loss: 0.0859 - acc: 1.0000 - val_loss: 0.3940 - val_acc: 0.8689
Epoch 45/200
1000/1000 [=======================

...

1000/1000 [==============================] - 1s 860us/step - loss: 0.0142 - acc: 1.0000 - val_loss: 0.3627 - val_acc: 0.8714
Epoch 150/200
1000/1000 [==============================] - 1s 852us/step - loss: 0.0140 - acc: 1.0000 - val_loss: 0.3628 - val_acc: 0.8714
Epoch 151/200
1000/1000 [==============================] - 1s 876us/step - loss: 0.0139 - acc: 1.0000 - val_loss: 0.3628 - val_acc: 0.8714
Epoch 152/200
1000/1000 [==============================] - 1s 856us/step - loss: 0.0137 - acc: 1.0000 - val_loss: 0.3628 - val_acc: 0.8713
Epoch 153/200
1000/1000 [==============================] - 1s 850us/step - loss: 0.0136 - acc: 1.0000 - val_loss: 0.3630 - val_acc: 0.8713
Epoch 154/200
1000/1000 [==============================] - 1s 846us/step - loss: 0.0134 - acc: 1.0000 - val_loss: 0.3629 - val_acc: 0.8715
Epoch 155/200
1000/1000 [==============================] - 1s 861us/step - loss: 0.0132 - acc: 1.0000 - val_loss: 0.3628 - val_acc: 0.8712
Epoch 156/200
1000/1000 [================

1000/1000 [==============================] - 1s 867us/step - loss: 0.0125 - acc: 1.0000 - val_loss: 0.3625 - val_acc: 0.8718
Epoch 161/200
1000/1000 [==============================] - 1s 866us/step - loss: 0.0123 - acc: 1.0000 - val_loss: 0.3626 - val_acc: 0.8715
Epoch 162/200
1000/1000 [==============================] - 1s 862us/step - loss: 0.0122 - acc: 1.0000 - val_loss: 0.3627 - val_acc: 0.8716
Epoch 163/200
1000/1000 [==============================] - 1s 862us/step - loss: 0.0120 - acc: 1.0000 - val_loss: 0.3627 - val_acc: 0.8716
Epoch 164/200
1000/1000 [==============================] - 1s 881us/step - loss: 0.0119 - acc: 1.0000 - val_loss: 0.3628 - val_acc: 0.8718
Epoch 165/200
1000/1000 [==============================] - 1s 862us/step - loss: 0.0118 - acc: 1.0000 - val_loss: 0.3628 - val_acc: 0.8716
Epoch 166/200
1000/1000 [==============================] - 1s 867us/step - loss: 0.0116 - acc: 1.0000 - val_loss: 0.3628 - val_acc: 0.8717
Epoch 167/200
1000/1000 [================

1000/1000 [==============================] - 1s 849us/step - loss: 0.0110 - acc: 1.0000 - val_loss: 0.3628 - val_acc: 0.8716
Epoch 172/200
1000/1000 [==============================] - 1s 855us/step - loss: 0.0109 - acc: 1.0000 - val_loss: 0.3628 - val_acc: 0.8719
Epoch 173/200
1000/1000 [==============================] - 1s 851us/step - loss: 0.0107 - acc: 1.0000 - val_loss: 0.3628 - val_acc: 0.8717
Epoch 174/200
1000/1000 [==============================] - 1s 880us/step - loss: 0.0106 - acc: 1.0000 - val_loss: 0.3628 - val_acc: 0.8716
Epoch 175/200
1000/1000 [==============================] - 1s 878us/step - loss: 0.0105 - acc: 1.0000 - val_loss: 0.3628 - val_acc: 0.8717
Epoch 176/200
1000/1000 [==============================] - 1s 853us/step - loss: 0.0104 - acc: 1.0000 - val_loss: 0.3630 - val_acc: 0.8716
Epoch 177/200
1000/1000 [==============================] - 1s 849us/step - loss: 0.0103 - acc: 1.0000 - val_loss: 0.3629 - val_acc: 0.8716
Epoch 178/200
1000/1000 [================

1000/1000 [==============================] - 1s 865us/step - loss: 0.0097 - acc: 1.0000 - val_loss: 0.3633 - val_acc: 0.8716
Epoch 183/200
1000/1000 [==============================] - 1s 871us/step - loss: 0.0096 - acc: 1.0000 - val_loss: 0.3632 - val_acc: 0.8718
Epoch 184/200
1000/1000 [==============================] - 1s 856us/step - loss: 0.0095 - acc: 1.0000 - val_loss: 0.3636 - val_acc: 0.8715
Epoch 185/200
1000/1000 [==============================] - 1s 870us/step - loss: 0.0094 - acc: 1.0000 - val_loss: 0.3632 - val_acc: 0.8712
Epoch 186/200
1000/1000 [==============================] - 1s 868us/step - loss: 0.0093 - acc: 1.0000 - val_loss: 0.3635 - val_acc: 0.8715
Epoch 187/200
1000/1000 [==============================] - 1s 862us/step - loss: 0.0092 - acc: 1.0000 - val_loss: 0.3635 - val_acc: 0.8715
Epoch 188/200
1000/1000 [==============================] - 1s 872us/step - loss: 0.0091 - acc: 1.0000 - val_loss: 0.3633 - val_acc: 0.8716
Epoch 189/200
1000/1000 [================

1000/1000 [==============================] - 1s 842us/step - loss: 0.0087 - acc: 1.0000 - val_loss: 0.3636 - val_acc: 0.8717
Epoch 194/200
1000/1000 [==============================] - 1s 862us/step - loss: 0.0086 - acc: 1.0000 - val_loss: 0.3635 - val_acc: 0.8718
Epoch 195/200
1000/1000 [==============================] - 1s 860us/step - loss: 0.0085 - acc: 1.0000 - val_loss: 0.3636 - val_acc: 0.8717
Epoch 196/200
1000/1000 [==============================] - 1s 898us/step - loss: 0.0084 - acc: 1.0000 - val_loss: 0.3635 - val_acc: 0.8718
Epoch 197/200
1000/1000 [==============================] - 1s 866us/step - loss: 0.0083 - acc: 1.0000 - val_loss: 0.3638 - val_acc: 0.8720
Epoch 198/200
1000/1000 [==============================] - 1s 841us/step - loss: 0.0082 - acc: 1.0000 - val_loss: 0.3640 - val_acc: 0.8720
Epoch 199/200
1000/1000 [==============================] - 1s 842us/step - loss: 0.0081 - acc: 1.0000 - val_loss: 0.3639 - val_acc: 0.8717
Epoch 200/200
1000/1000 [================